In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold

In [0]:
%sql

CREATE OR REPLACE TABLE gold.full_info AS
SELECT st.sensor_id, s.value, s.pollutant, s.unit, s.datetimeFrom as date, st.station_id, st.name,  st.latitude, st.longitude
FROM silver.sensors as s
INNER JOIN silver.station as st ON s.id = st.sensor_id


In [0]:
%sql
select count(*)
from gold.full_info

In [0]:
%sql

ALTER TABLE gold.full_info
ADD COLUMN hour int



In [0]:
%sql

ALTER TABLE gold.full_info
ADD COLUMN day int

In [0]:
%sql

ALTER TABLE gold.full_info
ADD COLUMN month int

In [0]:
%sql
UPDATE gold.full_info
SET day = EXTRACT(DAY FROM cast(date as timestamp)),
    hour = EXTRACT(HOUR FROM cast(date as timestamp)),
    month = EXTRACT(MONTH FROM cast(date as timestamp));

select *
from gold.full_info

In [0]:
%sql
Select *
from gold.full_info
order by month, day, hour


In [0]:
%sql
CREATE OR REPLACE VIEW gold.distribution AS
SELECT avg(value) as total_value,pollutant, name
FROM gold.full_info
WHERE pollutant not in ('relativehumidity', 'um003', 'temperature'  )  
Group by pollutant, name

In [0]:
%sql
Select *
from gold.distribution

In [0]:
%sql
CREATE OR REPLACE VIEW gold.timeline AS
Select ROUND(MAX(total_value)) as max_value, date_day, name, pollutant, month
from 
(SELECT SUM(value) as total_value, COUNT(distinct sensor_id) as total_sensors,month,  hour, day, date_trunc('day', cast(date as timestamp)) AS date_day, name, pollutant
FROM gold.full_info
WHERE pollutant not in ('relativehumidity', 'um003', 'temperature'  )
Group by name, pollutant, month, day, hour, date_day
order by name, pollutant, month,day, hour) as t
group by name, pollutant, date_day, month
order by date_day

In [0]:
%sql
Select *
from  gold.timeline